# 5장. XGBoost 소개

In [1]:
# 노트북이 코랩에서 실행 중인지 체크합니다.
import sys
if 'google.colab' in sys.modules:
    !pip install -q --upgrade xgboost
    !wget -q 

In [2]:
# 경고 끄기
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
xgb.set_config(verbosity=0)

## XGBoost 모델 만들기

### 붓꽃 데이터셋

In [3]:
import pandas as pd
import numpy as np
from sklearn import datasets
iris = datasets.load_iris()

In [4]:
df = pd.DataFrame(data=np.c_[iris['data'], iris['target']],
                  columns=iris['feature_names'] + ['target'])

In [5]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [6]:
from sklearn.model_selection import train_test_split

In [7]:
# 데이터를 훈련 세트와 테스트 세트로 나눕니다.
X_train, X_test, y_train, y_test = train_test_split(iris['data'], 
                                                    iris['target'], random_state=2)

#### XGBoost 분류 모델

In [8]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [9]:
xgb = XGBClassifier(booster='gbtree', objective='multi:softprob', 
                    max_depth=6, learning_rate=0.1, n_estimators=100, 
                    n_jobs=-1)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
score = accuracy_score(y_pred, y_test)
print('점수: ' + str(score))

점수: 0.9736842105263158


`accuracy_score()` 함수 대신 `score()` 메서드를 사용할 수 있습니다.

In [10]:
xgb.score(X_test, y_test)

0.9736842105263158

XGBoost의 기본 파이썬 API를 사용하는 경우 부스터(Booster) 객체의 `predict()` 메서드는 `multi:softprob`일 때 확률을 반환하고 `multi:softmax`일 때 클래스 레이블을 반환합니다.

In [11]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test[:5])

param = {'objective': 'multi:softprob', 'num_class': 3}
bstr = xgb.train(param, dtrain, 10)
bstr.predict(dtest)

array([[0.9486482 , 0.02711029, 0.02424142],
       [0.9486482 , 0.02711029, 0.02424142],
       [0.02841366, 0.05416913, 0.9174172 ],
       [0.9486482 , 0.02711029, 0.02424142],
       [0.9486482 , 0.02711029, 0.02424142]], dtype=float32)

In [12]:
param = {'objective': 'multi:softmax', 'num_class': 3}
bstr = xgb.train(param, dtrain, 10)
bstr.predict(dtest)

array([0., 0., 2., 0., 0.], dtype=float32)

### 당뇨병 데이터셋

In [13]:
X, y = datasets.load_diabetes(return_X_y=True)

#### XGBoost 회귀 모델 (교차 검증)

In [14]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor

xgb = XGBRegressor(booster='gbtree', objective='reg:squarederror', 
                   max_depth=6, learning_rate=0.1, n_estimators=100, 
                   n_jobs=-1)

scores = cross_val_score(xgb, X, y, 
                         scoring='neg_mean_squared_error', cv=5)

# 평가 점수의 제곱근을 계산한합니다.
rmse = np.sqrt(-scores)

# RMSE를 출력합니다.
print('RMSE:', np.round(rmse, 3))

# 평균 점수를 출력합니다.
print('RMSE 평균: %0.3f' % (rmse.mean()))

RMSE: [63.033 59.689 64.538 63.699 64.661]
RMSE 평균: 63.124


In [15]:
pd.DataFrame(y).describe()

,0
count,442.000000
mean,152.133484
std,77.093005
min,25.000000
25%,87.000000
50%,140.500000
75%,211.500000
max,346.000000


## 힉스 보손 찾기 - 사례연구

### 데이터

In [ ]:
# df = pd.read_csv('atlas-higgs-challenge-2014-v2.csv.gz', nrows=250000, compression='gzip')
df = pd.read_csv('https://raw.githubusercontent.com/rickiepark/handson-gb/main/Chapter05/atlas-higgs-challenge-2014-v2.csv.gz', nrows=250000, compression='gzip')
    
df.head()

In [ ]:
del df['Weight']
del df['KaggleSet']
df = df.rename(columns={"KaggleWeight": "Weight"})
label_col = df['Label']
del df['Label']
df['Label'] = label_col

##### 데이터 프레임의 열을 삭제하고 옮기는 방법

In [ ]:
# df_h = pd.read_csv('atlas-higgs-challenge-2014-v2.csv.gz', nrows=250000, compression='gzip')
df_h = pd.read_csv('https://raw.githubusercontent.com/rickiepark/handson-gb/main/Chapter05/atlas-higgs-challenge-2014-v2.csv.gz', nrows=250000, compression='gzip')


In [ ]:
df_new = df_h.drop(['Weight', 'KaggleSet', 'Label'], axis=1).assign(Label=df_h['Label'])
df_new = df_new.rename(columns={'KaggleWeight': 'Weight'})

In [ ]:
df_new.head()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['Label'].replace(('s', 'b'), (1, 0), inplace=True)

In [ ]:
X = df.iloc[:,1:31]
y = df.iloc[:,-1]

##### 열을 선택하는 다른 방법

In [ ]:
X_new = df_new.loc[:, ~df_new.columns.isin(['EventId','Weight','Label'])]
y_new = df_new.loc[:, 'Label']

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_new = le.fit_transform(df_new['Label'])
y_new

In [ ]:
print(le.classes_)

### 측정 지표

In [ ]:
xgb = XGBClassifier(n_estimators=5)
xgb.fit(X_train, y_train, 
        eval_set=[(X_train, y_train), (X_test, y_test)],
        eval_metric='mlogloss')

In [ ]:
xgb.evals_result()

### 가중치

In [ ]:
df['test_Weight'] = df['Weight'] * 550000 / len(y)

In [ ]:
s = np.sum(df[df['Label']==1]['test_Weight'])
b = np.sum(df[df['Label']==0]['test_Weight'])

In [ ]:
b/s

### 모델

In [ ]:
import xgboost as xgb

# 넘파이 배열로 xgboost.DMatrix 만들기. -999.0는 누락된 값으로 다룹니다.
xgmat = xgb.DMatrix(X, y, missing = -999.0, weight=df['test_Weight'])

# xgboost 파라미터 설정
param = {}
# 랭킹만 필요하기 때문에 로지스틱 함수를 적용하기 전의 출력을 사용합니다.
param['objective'] = 'binary:logitraw'
# 양성 샘플의 가중치를 조정합니다.
param['scale_pos_weight'] = b/s
param['eta'] = 0.1
param['max_depth'] = 6
param['eval_metric'] = 'auc'

# ams 지표를 추가합니다.
plst = list(param.items())+[('eval_metric', 'ams@0.15')]

watchlist = [(xgmat,'train')]

# 120개 트리 부스팅
num_round = 120

print('데이터 로딩 완료, 트리 부스팅 시작')
bst = xgb.train(plst, xgmat, num_round, watchlist)
bst.save_model('higgs.model')
print('훈련 종료')

##### 사이킷런 API로 구현하기

In [ ]:
clf = XGBClassifier(n_estimators=120, learning_rate=0.1, missing=-999.0, 
                    scale_pos_weight=b/s)

clf.fit(X, y, sample_weight=df['test_Weight'], 
        eval_set=[(X, y)], eval_metric=['auc', 'ams@0.15'],
        sample_weight_eval_set=[df['test_Weight']])

clf.save_model('higgs-sklearn.model')

In [ ]:
clf.evals_result()

In [ ]:
clf.score(X, y)